In [ ]:
# This reads in time-series data, bins it, and plots both

# Created 2021 Dept. 18 by E.S.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib qt

In [2]:
# read in time-series
# detrended_photom_0pt99_variance_0pt01_transit_1_col_star.csv
# detrended_photom_0pt98_variance_0pt01_off_center_transit_1_col_star.csv
# detrended_photom_0pt97_variance_0pt01_off_center_transit_1_col_star.csv
# detrended_photom_0pt85_variance_0pt01_off_center_transit_1_col_star.csv
# detrended_photom_0pt90_variance_0pt01_off_center_transit_1_col_star.csv

photometry_df = pd.read_csv("../detrended_photom_0pt85_variance_0pt01_off_center_transit_1_col_star.csv", 
                            index_col="Unnamed: 0")

# read in data with JD info and concatenate to photometry
time_df = pd.read_csv("test_time_series_20210917.csv", usecols=["jd-2459431"])
photometry_df = pd.concat([photometry_df,time_df], axis=1)

In [3]:
def error_bar_noise(data_pts):
    '''
    Generates error bars based on the number of data points in a bin
    
    INPUTS:
    data_pts: the data
    
    OUTPUTS:
    bar_length: the length of the error bar
    '''
    
    bar_length = np.divide(np.std(data_pts),np.sqrt(len(data_pts)))
    
    return bar_length

In [4]:
# configure binning

bin_size = 5*0.00069444444 # 60 sec is 0.00069444444 day
bin_edges = np.arange(start=np.min(time_df["jd-2459431"]),stop=np.max(time_df["jd-2459431"]),step=bin_size)
bin_centers = np.add(0.5*(bin_edges[1]-bin_edges[0]),bin_edges[:-1])
bin_labels = list(map(str,np.arange(0,len(bin_edges)-1)))

In [5]:
# put labels into a new column denoting which ranges of time are in which bin

photometry_df["time_bin"] = pd.cut(photometry_df["jd-2459431"], bins=bin_edges, labels=bin_labels, right=False)

In [6]:
# choose the photometry to use
# (note this is just a string for a column number, not a star number)
# IF YOU'RE TESTING RETRIEVAL OF FAKE PLANET, THIS STAR WILL NEED TO BE THE
# SAME AS THE ONE WITH THE TRANSIT

col_of_choice = "1"

In [7]:
# make new array of means and error bars
binned_photometry = []
binned_photometry_error_bars = []

for t in range(0,len(bin_labels)):
    
    # extract the datapoints corresponding to that value
    photometry_this_bin = photometry_df[col_of_choice].where(photometry_df["time_bin"] == bin_labels[t]).dropna()
    
    binned_photometry.append(np.mean(photometry_this_bin))
    binned_photometry_error_bars.append(error_bar_noise(photometry_this_bin))

In [19]:
plt.plot(photometry_df.median(axis=0))
plt.show()

<ipython-input-19-3611a39e7232>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  plt.plot(photometry_df.median(axis=0))


In [8]:
plt.clf()
plt.scatter(photometry_df["jd-2459431"],photometry_df[col_of_choice],s=2,alpha=0.5)
plt.errorbar(bin_centers,binned_photometry,yerr=binned_photometry_error_bars,linestyle="",ecolor="k",capsize=5)
#plt.ylim([7180,7280])
plt.xlabel("JD-2459431")
plt.show()
#plt.savefig("junk.pdf")

In [91]:
# fyi: typical percent precision we can get
'''
photom_prec = np.mean(np.divide(binned_photometry_error_bars,binned_photometry))
print(photom_prec)
'''

0.0051141664397614086